In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3,padding=1), # Input (1 * 28 * 28), Output (28 * 28 * 16)
            nn.ReLU(),
            nn.BatchNorm2d(16), # Input (28 * 28 * 16), Output (28 * 28 * 16)
            nn.Conv2d(16, 16, 3,padding=1), # Input (28 * 28 * 16), Output (28 * 28 * 16)
            nn.ReLU(),
            nn.BatchNorm2d(16), # Input (28 * 28 * 16), Output (28 * 28 * 16)
            nn.MaxPool2d(2, 2), # Input (28 * 28 * 16), Output (14 * 14 * 16)
            nn.Dropout(0.25)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 16, 3,padding=1), # Input (14 * 14 * 16), Output (14 * 14 * 16)
            nn.ReLU(),
            nn.BatchNorm2d(16), # Input (14 * 14 * 16), Output (14 * 14 * 16)
            nn.Conv2d(16, 16, 3,padding=1), # Input (14 * 14 * 16), Output (14 * 14 * 16)
            nn.ReLU(),
            nn.BatchNorm2d(16), #  Input (14 * 14 * 16), Output (14 * 14 * 16)
            nn.MaxPool2d(2, 2), # Input (14 * 14 * 16), Output (7 * 7 * 16)
            nn.Dropout(0.25)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 16, 3,padding=1), # Input (7 * 7 * 16), Output (7 * 7 * 16)
            nn.ReLU(),
            nn.BatchNorm2d(16), # Input (7 * 7 * 16), Output (7 * 7 * 16)
            nn.Conv2d(16, 32, 3,padding=1), # Input (7 * 7 * 16), Output (7* 7* 32)
            nn.ReLU(),
            nn.BatchNorm2d(32), #  Input (7 * 7 * 32), Output (7 * 7 * 32)

            nn.MaxPool2d(2, 2), # Input (7 * 7 * 32), Output (3 * 3 * 32)
            nn.Dropout(0.25)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(32, 10, 3,padding=1), # Input (3 * 3 * 32), Output (3 * 3 * 10)
            nn.ReLU(),
            nn.BatchNorm2d(10), # Input (7 * 7 * 16), Output (7 * 7 * 16)
            nn.AvgPool2d(2), # Input (3 * 3 * 10), Output (1 * 1 * 10)
            nn.Dropout(0.25)
        )
        
        self.linear = nn.Linear(in_features=1 * 1 * 10, out_features=10)

    def forward(self, x):
         x = self.conv1(x)
         x = self.conv2(x)
         x = self.conv3(x)
         x = self.conv4(x)
         x = x.view(-1, 1 * 1 * 10) 
         x = self.linear(x) 
         return F.log_softmax(x)
        

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
            Conv2d-4           [-1, 16, 28, 28]           2,320
              ReLU-5           [-1, 16, 28, 28]               0
       BatchNorm2d-6           [-1, 16, 28, 28]              32
         MaxPool2d-7           [-1, 16, 14, 14]               0
           Dropout-8           [-1, 16, 14, 14]               0
            Conv2d-9           [-1, 16, 14, 14]           2,320
             ReLU-10           [-1, 16, 14, 14]               0
      BatchNorm2d-11           [-1, 16, 14, 14]              32
           Conv2d-12           [-1, 16, 14, 14]           2,320
    

<ipython-input-16-9563cdaa3329>:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Building the network

In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
     

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(18):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-3-9563cdaa3329>:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.2614039182662964 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.77it/s]



Test set: Average loss: 0.0601, Accuracy: 9820/10000 (98%)



loss=0.2195345014333725 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.33it/s]



Test set: Average loss: 0.0412, Accuracy: 9867/10000 (99%)



loss=0.1670299768447876 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.84it/s]


...


Test set: Average loss: 0.0270, Accuracy: 9927/10000 (99%)



loss=0.01554136723279953 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.47it/s]



Test set: Average loss: 0.0215, Accuracy: 9937/10000 (99%)



loss=0.030078327283263206 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.06it/s]



Test set: Average loss: 0.0215, Accuracy: 9939/10000 (99%)

